<a href="https://colab.research.google.com/github/tlsgptj/CJONs/blob/test/%EC%99%80%EB%8C%95_%EB%AA%A8%EB%8D%B8_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 준비단계

## 1.0. 필요한 라이브러리 부르기

In [ ]:
%pip install numpy==1.23.4

from keras.preprocessing import sequence
#from sklearn.model_selection import train_test_split
# 여기 오류는 엘리스떄 처럼 넘파이 버전 오류임.
# 넘파이를 1.23.4 버전으로 바꾸니 성공.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 69.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


In [ ]:
!pip install wordcloud
!pip install transformers
!pip install transformers[torch]


# 필요한 라이브러리 불러오기
import pandas as pd
import urllib.request

# Bert 사용에 필요한 모듈 불러오기
import torch

import pandas as pd
import numpy as np
import random
import time
import datetime

from transformers import BertTokenizer
from transformers import AdamW, Adafactor, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

SEED = 42

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00


## 1.1. 파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path_test_dataset = '/content/Sea Smoke Southing Pinot Noir_cleaning.csv'
test_df = pd.read_csv(file_path_test_dataset)

test_df = test_df[~test_df['Review'].isnull()]
test_df.isnull().sum()

test_df.shape

(725, 3)

## 1.2. stopword

In [ ]:
#소문자로 바꾸고 쪼개기
import re
new_test_df =[]
for review in test_df['Review']:
    words = review.lower() # 소문자화

    # words = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\]','', words) # 문장 속 문장부호 지우기
    # words = review.split()

    new_test_df.append(words)

# # 필터링할 추가적인 stopwords
# new_test_df2 =[]
# stopwords =[
#             'incense', 'currant', 'coffee', 'allspice', 'strawberry',
#             'cigar', 'cassis', 'boysenberry', 'cola', 'oak', 'milk',
#             'blackcurrant', 'cotton', 'blackberry', 'berry', 'candy', 'clove',
#             'sandalwood', 'smoke', 'blueberry', 'ripe', 'raspberry', 'fresh', 'cherry', 'watermelon',
#             'espresso', 'butter', 'baking', 'bramble',  'olive', 'cranberry', 'camphor',
#             'pomegranate', 'pastry', 'bread', 'chocolate', 'coconut', 'mocha', 'cedar',
#             'fruit', 'toffee', 'tobacco', 'nutmeg', "fruity", "fruitful", "cherries", "blackberries", "vanilla"
#             ]


# def filter_stopwords(tokenized_text, stopwords_2):
#   tokenized_filtered = []

#   for i in tokenized_text:
#       if i not in stopwords and i is not None:
#         tokenized_filtered.append(i)

#   return tokenized_filtered

# for j in new_test_df:
#   tokenized_filtered = filter_stopwords(j, stopwords)
#   new_test_df2.append(tokenized_filtered)
# new_test_df = [inner_list for inner_list in new_test_df2 if any(inner_list)]

test_df['Review'] = new_test_df

## 1.3. Rating 값에 따라 전문가 / 초심자 분류

In [ ]:
df = test_df

threshold = 100     # 이 값에 따라 rating 자를 거임. 즉, 전문가 / 초심자 나누는 기준

# 'Rating' 값에 따라 분류
df_expert = df[df['Rating'] > threshold]
df_beginner = df[df['Rating'] <= threshold]

In [ ]:
df_expert.shape, df_beginner.shape

((450, 3), (275, 3))

# 2. 맛구별 시작

## 2.0. 모델 호출

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification      #여기서 Error 날 때 있는데, 그냥 다시 돌리면 실행됨;; 왜 그런지 모름;;

num_labels = 3  # 클래스 개수

# 맛 구별 모델 호출!!!!!!!!!!!!!
model_Sweet = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/와인 리뷰/bert 관련/bert_model_SD.h5')
model_Tannin = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/와인 리뷰/bert 관련/bert_model_ST.h5')
model_Acidic = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/와인 리뷰/bert 관련/bert_model_SA.h5')
model_Bold = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/와인 리뷰/bert 관련/bert_model_LB.h5')

# Tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some layers from the model checkpoint at /content/drive/MyDrive/와인 리뷰/bert 관련/bert_model_SD.h5 were not used when initializing TFBertForSequenceClassification: ['dropout_265']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/와인 리뷰/bert 관련/bert_model_SD.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without fu

## 2.1. 라이브러리 호출

In [ ]:
from copy import copy
import math

## 2.2. 단맛 와댕이

In [ ]:
# 추론
label_counts = [[0, 0, 0], [0, 0, 0]]  # 각 라벨 별 카운트 초기화
# [[0,     0,    0],   [0,     0,    0]]
# [[ex_un, ex_1, ex_2] [bg_un, bg_1, bg_2]]

for review in df_expert["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Sweet(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[0][predicted_label_id] += 1

for review in df_beginner["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Sweet(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[1][predicted_label_id] += 1

In [ ]:
# 라벨 1과 2의 비율 계산 및 출력
label_ex_count = label_counts[0]
label_bg_count = label_counts[1]

total_label_count_ex = label_ex_count[1] + label_ex_count[2]
total_label_count_bg = label_bg_count[1] + label_bg_count[2]

label_1_ratio_ex = label_ex_count[1] / total_label_count_ex * 100
label_2_ratio_ex = 100 - label_1_ratio_ex
label_1_ratio_bg = label_bg_count[1] / total_label_count_bg * 100
label_2_ratio_bg = 100 - label_1_ratio_bg
label_1_ratio_all = (label_ex_count[1] + label_bg_count[1]) / (total_label_count_ex + total_label_count_bg) * 100
label_2_ratio_all = 100 - label_1_ratio_all

print(f"sweet and dry Ratio (Expert)   : {label_1_ratio_ex:.2f} : {label_2_ratio_ex:.2f}")
print(f"sweet and dry Ratio (Beginner) : {label_1_ratio_bg:.2f} : {label_2_ratio_bg:.2f}")

print("expert : ",label_ex_count,"\nbeginner : ",label_bg_count)

sweet_ratio_ex = round(copy(label_1_ratio_ex))
dry_ratio_ex = round(copy(label_2_ratio_ex))
sweet_ratio_bg = round(copy(label_1_ratio_bg))
dry_ratio_bg = round(copy(label_2_ratio_bg))
sweet_ratio_all = round(copy(label_1_ratio_all))
dry_ratio_all = round(copy(label_2_ratio_all))

sweet and dry Ratio (Expert)   : 43.28 : 56.72
sweet and dry Ratio (Beginner) : 31.43 : 68.57
expert :  [383, 29, 38] 
beginner :  [240, 11, 24]


## 2.3. 쓴 맛 와댕이

In [ ]:
# 추론
label_counts = [[0, 0, 0], [0, 0, 0]]  # 각 라벨 별 카운트 초기화
# [[0,     0,    0],   [0,     0,    0]]
# [[ex_un, ex_1, ex_2] [bg_un, bg_1, bg_2]]

for review in df_expert["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Tannin(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[0][predicted_label_id] += 1

for review in df_beginner["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Tannin(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[1][predicted_label_id] += 1

In [ ]:
# 라벨 1과 2의 비율 계산 및 출력
label_ex_count = label_counts[0]
label_bg_count = label_counts[1]

total_label_count_ex = label_ex_count[1] + label_ex_count[2]
total_label_count_bg = label_bg_count[1] + label_bg_count[2]

label_1_ratio_ex = label_ex_count[1] / total_label_count_ex * 100
label_2_ratio_ex = 100 - label_1_ratio_ex
label_1_ratio_bg = label_bg_count[1] / total_label_count_bg * 100
label_2_ratio_bg = 100 - label_1_ratio_bg
label_1_ratio_all = (label_ex_count[1] + label_bg_count[1]) / (total_label_count_ex + total_label_count_bg) * 100
label_2_ratio_all = 100 - label_1_ratio_all

print(f"tannin and smooth Ratio (Expert)   : {label_1_ratio_ex:.2f} : {label_2_ratio_ex:.2f}")
print(f"tannin and smooth Ratio (Beginner) : {label_1_ratio_bg:.2f} : {label_2_ratio_bg:.2f}")

print("expert : ",label_ex_count,"\nbeginner : ",label_bg_count)

tannin_ratio_ex = round(copy(label_1_ratio_ex))
smooth_ratio_ex = round(copy(label_2_ratio_ex))
tannin_ratio_bg = round(copy(label_1_ratio_bg))
smooth_ratio_bg = round(copy(label_2_ratio_bg))
tannin_ratio_all = round(copy(label_1_ratio_all))
smooth_ratio_all = round(copy(label_2_ratio_all))

tannin and smooth Ratio (Expert)   : 24.28 : 75.72
tannin and smooth Ratio (Beginner) : 11.61 : 88.39
expert :  [277, 42, 131] 
beginner :  [163, 13, 99]


## 2.4. 신 맛 와댕이

In [ ]:
# 추론
label_counts = [[0, 0, 0], [0, 0, 0]]  # 각 라벨 별 카운트 초기화
# [[0,     0,    0],   [0,     0,    0]]
# [[ex_un, ex_1, ex_2] [bg_un, bg_1, bg_2]]

for review in df_expert["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Acidic(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[0][predicted_label_id] += 1

for review in df_beginner["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Acidic(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[1][predicted_label_id] += 1


In [ ]:
# 라벨 1과 2의 비율 계산 및 출력
label_ex_count = label_counts[0]
label_bg_count = label_counts[1]

total_label_count_ex = label_ex_count[1] + label_ex_count[2]
total_label_count_bg = label_bg_count[1] + label_bg_count[2]

label_1_ratio_ex = label_ex_count[1] / total_label_count_ex * 100
label_2_ratio_ex = 100 - label_1_ratio_ex
label_1_ratio_bg = label_bg_count[1] / total_label_count_bg * 100
label_2_ratio_bg = 100 - label_1_ratio_bg
label_1_ratio_all = (label_ex_count[1] + label_bg_count[1]) / (total_label_count_ex + total_label_count_bg) * 100
label_2_ratio_all = 100 - label_1_ratio_all

print(f"acidic and soft Ratio (Expert)   : {label_1_ratio_ex:.2f} : {label_2_ratio_ex:.2f}")
print(f"acidic and soft Ratio (Beginner) : {label_1_ratio_bg:.2f} : {label_2_ratio_bg:.2f}")

print("expert : ",label_ex_count,"\nbeginner : ",label_bg_count)

acidic_ratio_ex = round(copy(label_1_ratio_ex))
soft_ratio_ex = round(copy(label_2_ratio_ex))
acidic_ratio_bg = round(copy(label_1_ratio_bg))
soft_ratio_bg = round(copy(label_2_ratio_bg))
acidic_ratio_all = round(copy(label_1_ratio_all))
soft_ratio_all = round(copy(label_2_ratio_all))

acidic and soft Ratio (Expert)   : 52.94 : 47.06
acidic and soft Ratio (Beginner) : 32.89 : 67.11
expert :  [263, 99, 88] 
beginner :  [199, 25, 51]


## 2.5. 바디감

In [ ]:
# 추론
label_counts = [[0, 0, 0], [0, 0, 0]]  # 각 라벨 별 카운트 초기화
# [[0,     0,    0],   [0,     0,    0]]
# [[ex_un, ex_1, ex_2] [bg_un, bg_1, bg_2]]

for review in df_expert["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Bold(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[0][predicted_label_id] += 1

for review in df_beginner["Review"]:

    # 토큰화 및 인코딩
    inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors="tf", padding=True, truncation=True)

    # attention_mask 생성 (패딩 토큰도 고려)
    attention_mask = tf.where(inputs["input_ids"] != tokenizer.pad_token_id, 1, 0)

    # 추론
    outputs = model_Bold(inputs)

    # 가장 높은 확률 값을 가진 클래스 선택
    predicted_label_id = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    # 해당 클래스의 카운트 증가
    label_counts[1][predicted_label_id] += 1


In [ ]:
# 라벨 1과 2의 비율 계산 및 출력
label_ex_count = label_counts[0]
label_bg_count = label_counts[1]

total_label_count_ex = label_ex_count[1] + label_ex_count[2]
total_label_count_bg = label_bg_count[1] + label_bg_count[2]

label_1_ratio_ex = label_ex_count[1] / total_label_count_ex * 100
label_2_ratio_ex = 100 - label_1_ratio_ex
label_1_ratio_bg = label_bg_count[1] / total_label_count_bg * 100
label_2_ratio_bg = 100 - label_1_ratio_bg
label_1_ratio_all = (label_ex_count[1] + label_bg_count[1]) / (total_label_count_ex + total_label_count_bg) * 100
label_2_ratio_all = 100 - label_1_ratio_all

print(f"light and bold Ratio (Expert)   : {label_1_ratio_ex:.2f} : {label_2_ratio_ex:.2f}")
print(f"light and bold Ratio (Beginner) : {label_1_ratio_bg:.2f} : {label_2_ratio_bg:.2f}")

print("expert : ",label_ex_count,"\nbeginner : ",label_bg_count)

light_ratio_ex = round(copy(label_1_ratio_ex))
bold_ratio_ex = round(copy(label_2_ratio_ex))
light_ratio_bg = round(copy(label_1_ratio_bg))
bold_ratio_bg = round(copy(label_2_ratio_bg))
light_ratio_all = round(copy(label_1_ratio_all))
bold_ratio_all = round(copy(label_2_ratio_all))

light and bold Ratio (Expert)   : 61.97 : 38.03
light and bold Ratio (Beginner) : 47.06 : 52.94
expert :  [308, 88, 54] 
beginner :  [207, 32, 36]


## 2.6. 데이터 모아보기

In [ ]:
print("Expert Review Flavor\n")
print("Sweet and Dry Ratio           ",  sweet_ratio_ex, " : ", dry_ratio_ex)
print("Tannins and Smooth Ratio      ",  tannin_ratio_ex, " : ", smooth_ratio_ex)
print("Acidic and Soft Ratio         ",  acidic_ratio_ex, " : ", soft_ratio_ex)
print("Light and Bold Ratio          ",  light_ratio_ex, " : ", bold_ratio_ex)

print('\n')

print("Beginner Review Flavor\n")
print("Sweet and Dry Ratio           ",  sweet_ratio_bg, " : ", dry_ratio_bg)
print("Tannins and Smooth Ratio      ",  tannin_ratio_bg, " : ", smooth_ratio_bg)
print("Acidic and Soft Ratio         ",  acidic_ratio_bg, " : ", soft_ratio_bg)
print("Light and Bold Ratio          ",  light_ratio_bg, " : ", bold_ratio_bg)

print('\n')

print("All Review Flavor\n")
print("Sweet and Dry Ratio           ",  sweet_ratio_all, " : ", dry_ratio_all)
print("Tannins and Smooth Ratio      ",  tannin_ratio_all, " : ", smooth_ratio_all)
print("Acidic and Soft Ratio         ",  acidic_ratio_all, " : ", soft_ratio_all)
print("Light and Bold Ratio          ",  light_ratio_all, " : ", bold_ratio_all)

Expert Review Flavor

Sweet and Dry Ratio            43  :  57
Tannins and Smooth Ratio       24  :  76
Acidic and Soft Ratio          53  :  47
Light and Bold Ratio           62  :  38


Beginner Review Flavor

Sweet and Dry Ratio            31  :  69
Tannins and Smooth Ratio       12  :  88
Acidic and Soft Ratio          33  :  67
Light and Bold Ratio           47  :  53


All Review Flavor

Sweet and Dry Ratio            39  :  61
Tannins and Smooth Ratio       19  :  81
Acidic and Soft Ratio          47  :  53
Light and Bold Ratio           57  :  43


# 3. 키워드분석

In [ ]:
!pip install wordcloud
!pip install transformers

# 필요한 라이브러리 불러오기
import pandas as pd
import urllib.request

In [ ]:
# 1. 필요한 모듈 불러오기
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from matplotlib import rc
import re

In [ ]:
# 2. Stopwords 및 특수문자 필터링을 위한 준비

comment_words = ''
stopwords = set(STOPWORDS)

tokenized_text = []

# 3. 문장 데이터를 단어화하기
for val in df["Review"]:

  # 문장을 string으로 만들기
  val = str(val)

  # 문장을 소문자로 바꾸기
  val = val.lower()

  #문장 속 문장부호 지우기
  val = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\]','', val)

  # 문장을 쪼개기
  tokens = val.split()

  tokenized_text.append(tokens)
  comment_words += " ".join(tokens) + " "

wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)

In [ ]:
# 필터링할 추가적인 stopwords들 정의

stopwords_Rose = ['wine', 'good', 'nice', 'nose', 'really', 'first', 'great', 'little',
                  'high', 'easy', 'top', 'color', 'long', 'tasty', 'excellent', 'very', 'notes',
                  'taste', 'colour', 'still', 'one', 'well', 'perfect', 'drink', 'aftertaste',
                  'value'
]
stopwords_name = ['provence'
]

In [ ]:
def filter_stopwords(tokenized_text, stopwords_2, stopword_3):
  tokenized_filtered = []

  for i in tokenized_text:
    for word in i:
      if word not in stopwords and word not in stopwords_2 and word not in stopword_3:
        tokenized_filtered.append(word)

  return tokenized_filtered

tokenized_filtered = filter_stopwords(tokenized_text, stopwords_Rose, stopwords_name)

In [ ]:
import operator

def word_count(tokenized_data):
  word_counter = {}

  for i in tokenized_data:
    if i in word_counter.keys():
      word_counter[i] += 1
    else:
      word_counter[i] = 1

  # 많이 나온 순서대로 정렬

  sorted_dict = dict( sorted(word_counter.items(),
                           key=operator.itemgetter(1), reverse=True))

  return sorted_dict

In [ ]:
tokenized_dict = word_count(tokenized_filtered)

In [ ]:
# Dry-Sweet 지표가 잘 나타난 것 구하기

import re

# 단어 분류 기준

flavor_dict_sweet_dry = {"dry": "Dry", "astringent": "Dry", "lean": "Dry", "crisp": "Dry", "tart": "Dry",
               "off-dry": "Dry", "bone-dry": "Dry", "stony": "Dry", "dusty": "Dry", "dried": "Dry", "dryness": "Dry",
               "sweet": "Sweet", "sweetness": "Sweet", "sugary": "Sweet", "sweetish": "Sweet",
               "honeyed": "Sweet", "luscious": "Sweet", "nectar-like": "Sweet", "sweety": "Sweet", "desert": "Sweet"
}
flavor_dict_tannin_smooth = {
    "astringency": "Tannins", "austere": "Tannins", "bitterness": "Tannins",
    "hard": "Tannins", "harsh": "Tannins", "tannins": "Tannins", "tannin": "Tannins",
    "structured": "Tannins", "grippy": "Tannins","tannic": "Tannins", "smooth": "Smooth",
    "silky": "Smooth", "velvety": "Smooth", "elegant": "Smooth",
    "round": "Smooth", "supple": "Smooth", "luscious": "Smooth",
    "balanced": "Smooth"
}
flavor_dict_acidic_soft = {
    "tart": "Acidic", "crisp": "Acidic", "zesty": "Acidic", "vinegary": "Acidic",
    "lively": "Acidic", "bright": "Acidic", "tangy": "Acidic", 'acid': "Acidic",
    "grippy": "Acidic", "hard": "Acidic", "acidic": "Acidic", 'acidity': "Acidic",
    "harsh": "Acidic" , "austere": "Acidic","sour":"Acidic","pungent":"Acidic","acidulous":"Acidic",

    "silky": "Soft", "supple": "Soft", "velvety": "Soft",
    "elegant": "Soft", "round": "Soft", "luscious": "Soft",
    "balanced": "Soft","softness":"Soft", "soft": "Soft",
    "softly": "Soft", "soften": "Soft", "softness": "Soft", "softy": "Soft",
}
flavor_dict_Light_Bold = {
    "full-bodied": "Bold", "bodied": "Bold", "robust": "Bold", "bold": "Bold", "rich": "Bold",
    "full-flavored": "Bold", "intense": "Bold", "weighty": "Bold", "concentrated": "Bold", "weight": "Bold", "boldness": "Bold",

    "light": "Light", "light-bodied": "Light", "ethereal": "Light", "delicate": "Light",
    "luscious": "Light", "unburdened": "Light", "lighty": "Light", "feathery": "Light",
    "airy": "Light", "mild": "Light", "graceful": "Light", "easygoing": "Light"
}

In [ ]:
from collections import defaultdict, OrderedDict

flavors_dict_SWEET = defaultdict(int)
flavors_dict_Tannins = defaultdict(int)
flavors_dict_Acidic = defaultdict(int)
flavors_dict_Light = defaultdict(int)

flavors_list_SWEET = []
flavors_list_Tannins = []
flavors_list_Acidic = []
flavors_list_Light = []

#단맛 키워드 기반 감정분석

for k, v in tokenized_dict.items():
  for key, value in flavor_dict_sweet_dry.items():
    if k == key:
      flavors_list_SWEET.append((value, v))

for k, v in flavors_list_SWEET:
  if k in flavors_dict_SWEET:
    flavors_dict_SWEET[k] += v
  else:
    flavors_dict_SWEET[k] = v

flavors_dict_SWEET = OrderedDict(sorted(flavors_dict_SWEET.items(),
                            key=lambda item: item[1],
                            reverse=True))

print("flavors_dict_SWEET : ", flavors_dict_SWEET)


#쓴맛 키워드 기반 감정분석

for k, v in tokenized_dict.items():
  for key, value in flavor_dict_tannin_smooth.items():
    if k == key:
      flavors_list_Tannins.append((value, v))

for k, v in flavors_list_Tannins:
  if k in flavors_dict_Tannins:
    flavors_dict_Tannins[k] += v
  else:
    flavors_dict_Tannins[k] = v

flavors_dict_Tannins = OrderedDict(sorted(flavors_dict_Tannins.items(),
                            key=lambda item: item[1],
                            reverse=True))

print("flavors_dict_Tannins : ", flavors_dict_Tannins)


#신맛 키워드 기반 분석

for k, v in tokenized_dict.items():
  for key, value in flavor_dict_acidic_soft.items():
    if k == key:
      flavors_list_Acidic.append((value, v))

for k, v in flavors_list_Acidic:
  if k in flavors_dict_Acidic:
    flavors_dict_Acidic[k] += v
  else:
    flavors_dict_Acidic[k] = v

flavors_dict_Acidic = OrderedDict(sorted(flavors_dict_Acidic.items(),
                            key=lambda item: item[1],
                            reverse=True))

print("flavors_dict_Acidic : ", flavors_dict_Acidic)


#바디감 키워드 기반 분석

for k, v in tokenized_dict.items():
  for key, value in flavor_dict_Light_Bold.items():
    if k == key:
      flavors_list_Light.append((value, v))

for k, v in flavors_list_Light:
  if k in flavors_dict_Light:
    flavors_dict_Light[k] += v
  else:
    flavors_dict_Light[k] = v

flavors_dict_Light = OrderedDict(sorted(flavors_dict_Light.items(),
                            key=lambda item: item[1],
                            reverse=True))

print("flavors_dict_Light : ", flavors_dict_Light)

flavors_dict_SWEET :  OrderedDict([('Dry', 54), ('Sweet', 27)])
flavors_dict_Tannins :  OrderedDict([('Smooth', 232), ('Tannins', 92)])
flavors_dict_Acidic :  OrderedDict([('Acidic', 129), ('Soft', 122)])
flavors_dict_Light :  OrderedDict([('Light', 79), ('Bold', 77)])


## 3.1. 그림으로 나타내기

In [ ]:
# Plotly로 시각화

# Pie Chart
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

flavors = pd.Series(flavors_dict_SWEET)

# 세 개의 플롯을 한 화면에 나타내기
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# 서브플롯 생성
fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]])

# Sweet - Dry 플롯
fig1 = px.pie(values=list(flavors_dict_SWEET.values()), names=list(flavors_dict_SWEET.keys()), title='Sweet - Dry')
fig1.update_traces(marker=dict(colors=['Pink', 'skyBlue']))

fig1.show()

# Tannins - Smooth 플롯
fig2 = px.pie(values=list(flavors_dict_Tannins.values()), names=list(flavors_dict_Tannins.keys()), title='Tannins - Smooth')
fig2.update_traces(marker=dict(colors=['red', 'blue']))

fig2.show()

# Acidic - Soft 플롯
fig3 = px.pie(values=list(flavors_dict_Acidic.values()), names=list(flavors_dict_Acidic.keys()), title='Acidic - Soft')
fig3.update_traces(marker=dict(colors=['green', 'Yellow']))

fig3.show()

# Light - Bold 플롯
fig4 = px.pie(values=list(flavors_dict_Light.values()), names=list(flavors_dict_Light.keys()), title='Light - Bold')
fig4.update_traces(marker=dict(colors=['black', 'grey']))

fig4.show()

In [ ]:
flavor1, flavor2 = list(flavors_dict_SWEET.values())
sweet_key = round(copy(flavor2 / (flavor1+flavor2) * 100))     #sweet 2번

flavor1, flavor2 = list(flavors_dict_Tannins.values())
tannins_key = round(copy(flavor2 / (flavor1+flavor2) * 100))   #tannin 2번

flavor1, flavor2 = list(flavors_dict_Acidic.values())
acidic_key = round(copy(flavor1 / (flavor1+flavor2) * 100))    #acidic 1번

flavor1, flavor2 = list(flavors_dict_Light.values())
light_key = round(copy(flavor1 / (flavor1+flavor2) * 100))     #light 1번

# 4. 데이터셋 만들기

## 4.0. 저장할 데이터셋 csv로 받기

In [ ]:
df_Dataset = pd.read_csv('/content/WineDataset.csv')


## 4.1. 데이터셋에 필요한 데이터들 추가하기

In [ ]:
df_tmp = pd.DataFrame({
    "WineName": "Sea Smoke Southing Pinot Noir",  #이름 바꿔주기
    "Sweet_ex": [sweet_ratio_ex],
    "Tannins_ex": [tannin_ratio_ex],
    "Acidic_ex": [acidic_ratio_ex],
    "Light_ex": [light_ratio_ex],
    "Sweet_bg": [sweet_ratio_bg],
    "Tannins_bg": [tannin_ratio_bg],
    "Acidic_bg": [acidic_ratio_bg],
    "Light_bg": [light_ratio_bg],
    "Sweet_all": [sweet_ratio_all],
    "Tannins_all": [tannin_ratio_all],
    "Acidic_all": [acidic_ratio_all],
    "Light_all": [light_ratio_all],
    "Sweet_key": [sweet_key],
    "Tannins_key": [tannins_key],
    "Acidic_key": [acidic_key],
    "Light_key": [light_key]
})

In [ ]:
df_Dataset = df_Dataset.append(df_tmp, ignore_index = True)

<ipython-input-400-45b4f3f6052c>:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
df_Dataset

,WineName,Sweet_ex,Tannins_ex,Acidic_ex,Light_ex,Sweet_bg,Tannins_bg,Acidic_bg,Light_bg,Sweet_all,...,Light_all,Sweet_key,Tannins_key,Acidic_key,Light_key,Sweet_vi,Tannins_vi,Acidic_vi,Light_vi,Tag
0,Torre Oria Knock Knock Red Blend,54,32,61,77,55,13,69,90,55,...,83,46,32,57,68,NaN,NaN,NaN,NaN,NaN
1,Weinert Carrascal 2002,21,47,42,39,23,41,38,34,22,...,38,28,47,58,85,NaN,NaN,NaN,NaN,NaN
2,Bastide Miraflors Vieilles Vignes Syrah - Gren...,23,42,56,49,47,20,43,50,27,...,49,18,48,59,72,NaN,NaN,NaN,NaN,NaN
3,Château Pibran Pauillac,16,52,45,40,22,23,50,47,18,...,42,14,50,55,74,NaN,NaN,NaN,NaN,NaN
4,Domain Road Vineyard Pinot Noir 2020,23,47,62,64,40,21,29,56,28,...,62,31,49,60,51,NaN,NaN,NaN,NaN,NaN
5,Dominus Dominus (Christian Moueix),40,30,24,39,45,27,17,30,40,...,38,44,43,68,78,NaN,NaN,NaN,NaN,NaN
6,Duckhorn Three Palms Vineyard Merlot,33,29,34,40,25,14,35,41,30,...,40,26,33,67,74,NaN,NaN,NaN,NaN,NaN
7,Mysterious Diggings Pinot Noir 2022,24,40,67,69,48,12,60,72,38,...,70,41,41,61,61,NaN,NaN,NaN,NaN,NaN
8,River Run Pinot Noir 2021,39,32,53,78,22,18,54,73,36,...,76,26,37,52,75,NaN,NaN,NaN,NaN,NaN
9,Rutherford Hill Merlot,36,29,54,46,36,15,39,55,36,...,50,25,30,58,65,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Dataset.to_csv('WineDataset.csv', index=False)